# AI Red Teaming Notebook Overview

This streamlined notebook guides you through progressively richer AI red teaming evaluations using the Azure AI Evaluation SDK.

You will:
- Run a fast smoke test with a deterministic safe callback (baseline expectations, near-zero Attack Success Rate).
- Target a real Azure OpenAI deployment to observe genuine safety behavior.
- Expand coverage across multiple risk categories and layered attack strategies.
- Add advanced multi-strategy scans (including composed transformations) to probe layered defenses.
- (Optional, end of notebook) Supply your own domain‑specific risky objectives.

Artifacts: Each scan writes a JSON scorecard file (label + UTC time). Use these for comparison, regression tracking, or upload into Azure AI Foundry.

Execution time scales roughly with: risk_categories × attack_strategies × num_objectives. Start small, expand only after verifying prior steps.

In [1]:
# Cell 1: Installation
import sys, subprocess

packages = [
    "duckdb==1.3.2",
    "azure-ai-evaluation[redteam]",
    "azure-identity",
    "openai",
    "azure-ai-projects",
    "python-dotenv",
]

subprocess.check_call([
    sys.executable,
    "-m",
    "pip",
    "install",
    "--quiet",
    "--upgrade",
    "--no-warn-conflicts",
    *packages,
])
print("Installed (pinned / required):")
for p in packages:
    print("  -", p)

Installed (pinned / required):
  - duckdb==1.3.2
  - azure-ai-evaluation[redteam]
  - azure-identity
  - openai
  - azure-ai-projects
  - python-dotenv


In [2]:
# Cell 2 - imports
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI import
from openai import AzureOpenAI

## Core Concepts: RedTeam, Risk Categories, Attack Strategies & Targets

**RedTeam Orchestrator**: Generates attack objectives, transforms prompts via strategies, invokes your target, and scores responses.

**Risk Categories (what we probe)**: Violence, Hate/Unfairness, Sexual, SelfHarm. You can supply a subset for faster iteration. Missing categories reduce coverage but cut cost/time.

**Attack Strategies (how we probe)**:
- Complexity group macros: `EASY`, `MODERATE` (bundles of simpler / moderate transformations)
- Individual transformations: Flip, CharSwap, UnicodeConfusable, Leetspeak, Url, Base64, ROT13, etc.
- Composition: `AttackStrategy.Compose([Base64, ROT13])` layers transformations to simulate obfuscation chains.

**num_objectives**: Count of seed prompts per category (per applied strategy). Linear multiplier on runtime.

**Targets (what gets attacked)**:
1. Simple synchronous callback (returns fixed text) – deterministic baseline.
2. Model configuration dict – RedTeam handles generation calls internally.
3. Fully custom (async) application wrapper – replicate real app logic, pre/post-processing.

We progress through (1) → (2) → (3+) for clarity.

> Tip: Keep early scans lean (≤2 categories, 1 strategy, num_objectives=1) to validate authentication & environment quickly.

In [3]:
# Cell 3 - Create credential (switchable auth)
import os
from azure.identity import DefaultAzureCredential, ManagedIdentityCredential

# Allow easy switching via env flag; default to DefaultAzureCredential for broader token chain
_use_default = os.environ.get("REDTEAM_USE_DEFAULT_CRED", "1") == "1"
if _use_default:
    credential = DefaultAzureCredential(exclude_interactive_browser_credential=True)
    print("Using DefaultAzureCredential (Managed Identity + Azure CLI + Env, etc.)")
else:
    credential = ManagedIdentityCredential()
    print("Using ManagedIdentityCredential explicitly")

# Quick probe (optional) – will no-op on some identities if scope isn't accessible
try:
    token = credential.get_token("https://management.azure.com/.default")
    print("Acquired mgmt token (truncated):", token.token[:24], "...")
except Exception as e:  # noqa: BLE001
    print("Token probe skipped:", e)


Using DefaultAzureCredential (Managed Identity + Azure CLI + Env, etc.)
Acquired mgmt token (truncated): eyJ0eXAiOiJKV1QiLCJhbGci ...


In [4]:
# Cell 4 - Dynamic .env discovery
from pathlib import Path
from typing import List
import os, re
from dotenv import load_dotenv

REQUIRED_KEYS = [
    "AZURE_SUBSCRIPTION_ID",
    "AZURE_RESOURCE_GROUP_NAME",
    "AZURE_PROJECT_NAME",
    "AZURE_OPENAI_DEPLOYMENT_NAME",
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
]

explicit_path = os.environ.get("REDTEAM_DOTENV_PATH")
searched: List[Path] = []
selected = None

candidates: List[Path] = []
if explicit_path:
    p = Path(explicit_path)
    if p.is_file():
        candidates.append(p)

# Typical Azure AI Foundry project mount pattern: /afh/projects/<resource-project-guid>/shared/files/.env
root = Path('/afh/projects')
if root.is_dir():
    for child in root.iterdir():
        if child.is_dir() and ('-project-' in child.name):
            env_candidate = child / 'shared' / 'files' / '.env'
            searched.append(env_candidate)
            if env_candidate.is_file():
                candidates.append(env_candidate)

# Fallback: shallow glob for any .env directly under shared/files
if not candidates and root.is_dir():
    for env_candidate in root.glob('**/shared/files/.env'):
        searched.append(env_candidate)
        if env_candidate.is_file():
            candidates.append(env_candidate)
            break

# Choose first containing all required keys, else first existing
for c in candidates:
    try:
        text = c.read_text()
        if all(re.search(rf'^ {k}=', text, re.MULTILINE) or re.search(rf'^{k}=', text, re.MULTILINE) for k in REQUIRED_KEYS):
            selected = c
            break
    except Exception:
        pass
if selected is None and candidates:
    selected = candidates[0]

if selected and selected.is_file():
    load_dotenv(selected)
    missing_after = [k for k in REQUIRED_KEYS if not os.environ.get(k)]
    print(f"Loaded .env from: {selected}")
    if missing_after:
        print("Still missing keys:", missing_after)
else:
    print("No .env loaded. Candidates searched (first 5):", [str(p) for p in searched[:5]])


Loaded .env from: /afh/projects/airt15-project-45f916a5-0d22-4510-bd9c-768b55d93670/shared/files/.env


In [5]:
# Cell 5 - Set variables
import os

_required_keys = [
    "AZURE_SUBSCRIPTION_ID",
    "AZURE_RESOURCE_GROUP_NAME",
    "AZURE_PROJECT_NAME",
    "AZURE_OPENAI_DEPLOYMENT_NAME",
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
]
_env = {k: os.environ.get(k) for k in _required_keys}
_missing = [k for k, v in _env.items() if not v]

if _missing:
    print("Missing environment variables:", _missing)
else:
    # Construct objects / variables consumed by later cells
    azure_ai_project = {
        "subscription_id": _env["AZURE_SUBSCRIPTION_ID"],
        "resource_group_name": _env["AZURE_RESOURCE_GROUP_NAME"],
        "project_name": _env["AZURE_PROJECT_NAME"],
        "credential": credential,
    }
    azure_openai_deployment = _env["AZURE_OPENAI_DEPLOYMENT_NAME"]
    azure_openai_endpoint = _env["AZURE_OPENAI_ENDPOINT"]
    azure_openai_api_key = _env["AZURE_OPENAI_API_KEY"]
    azure_openai_api_version = _env["AZURE_OPENAI_API_VERSION"]

    print("Config loaded.")
    print(f"Deployment: {azure_openai_deployment}")
    print(f"Endpoint: {azure_openai_endpoint}")
    print("API key prefix:", azure_openai_api_key[:6] + '...')

Config loaded.
Deployment: gpt-4o-mini
Endpoint: https://airt15aibsntti.openai.azure.com/
API key prefix: 50de11...


### Environment Variables Loaded
Required keys (subscription, resource group, project name, OpenAI deployment details, API key & version) enable two things:
1. Locating the Azure AI Project for logging / result persistence.
2. Invoking your Azure OpenAI deployment during model-based scans.

Missing values will be surfaced in the next cell; resolve them before proceeding to model or advanced scans.

### Target Patterns
You can point RedTeam at different target styles:
- Deterministic callback (fast baseline, isolates framework logic).
- Direct model configuration dict (minimal code, invokes deployment directly).
- Rich application function (mirrors real system behavior: pre/post-processing, memory, safety layers).

We start with the deterministic callback to validate plumbing before spending tokens on real model invocations.

In [6]:
# Cell 6 - Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [7]:
# Cell 7 - Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
 )
# (Inserted) Configure writable output directory to avoid permission issues in read-only code mount
import pathlib, os
_rt_out = pathlib.Path.home() / 'redteam_outputs'
_rt_out.mkdir(parents=True, exist_ok=True)
try:
    # Some internals may expose file_manager; guard if attribute layout changes
    if hasattr(red_team, 'file_manager') and hasattr(red_team.file_manager, 'base_output_dir'):
        red_team.file_manager.base_output_dir = str(_rt_out)
        print('RedTeam base_output_dir set to', red_team.file_manager.base_output_dir)
    else:
        print('WARNING: Unable to set custom base_output_dir; attribute not found')
except Exception as _e:
    print('WARNING: Failed to set base_output_dir:', _e)

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


RedTeam base_output_dir set to /home/azureuser/redteam_outputs


In [8]:
# Cell 8 - Basic Scan
from datetime import datetime, timezone
from pathlib import Path
import pathlib, os

scan_ts = datetime.now(timezone.utc).strftime("%H%M%S")
scan_label = "Basic"
scan_name = f"{scan_label}-Scan-{scan_ts}"
# Use writable output directory set earlier (fallback to home)
_writable_base = pathlib.Path(getattr(red_team.file_manager, 'base_output_dir', Path.home()))
_writable_base.mkdir(parents=True, exist_ok=True)
output_path = _writable_base / f"{scan_label.lower()}_scan-{scan_ts}.json"

basic_scan_result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name=scan_name,
    attack_strategies=[AttackStrategy.Flip],
    output_path=str(output_path),
    # skip_upload=True  # uncomment if remote artifact upload warnings persist
 )
print(f"Completed {scan_name} -> {output_path}")

🚀 STARTING RED TEAM SCAN
📂 Output directory: /home/azureuser/redteam_outputs/.scan_Basic-Scan-204708_20251016_204708
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/14a946ff-9d0c-485e-a308-039a4a1f29f2?wsid=/subscriptions/e440a65b-7418-4865-9821-88e411ffdd5b/resourceGroups/AIRT15/providers/Microsoft.MachineLearningServices/workspaces/AIRT15-project
📋 Planning 4 total tasks
📝 Fetched baseline objectives for violence: 1 objectives
📝 Fetched baseline objectives for hate_unfairness: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


Scanning:   0%|                                       | 0/4 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 4 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: flip strategy for violence risk category
▶️ Starting task: flip strategy for hate_unfairness risk category


Scanning: 100%|███████████████████████████████| 4/4 [00:07<00:00,  1.77s/scan, current=initializing]


Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Basic-Scan-204708_20251016_204708/flip_violence_86d73dda-7d7e-4a99-9fb1-435e54cc9a0a.json".
✅ Completed task 1/4 (25.0%) - flip/violence in 6.9s
   Est. remaining: 0.5 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Basic-Scan-204708_20251016_204708/flip_hate_unfairness_b0312a8b-8833-464b-bad0-94bba892fe06.json".
✅ Completed task 2/4 (50.0%) - flip/hate_unfairness in 7.0s
   Est. remaining: 0.2 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Basic-Scan-204708_20251016_204708/baseline_hate_unfairness_1632e49c-a623-4215-8137-5081dd7a6a7a.json".
✅ Completed task 3/4 (75.0%) - baseline/hate_unfairness in 7.0s
   Est. remaining: 0.1 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Basic-Scan-204708_20251016_204708/baseline_violence_8cb96946-a1c6-434d-8ce9-cd20999252f3.json".
✅ Completed task 4/4 (100.0%) - baseline/violence in 7.1s
   Est. remain

Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "/home/azureuser/redteam_outputs/basic_scan-204708.json".

Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Basic-Scan-204708_20251016_204708/final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence             | 0.0%           | 0.0%                         | N/A                             | N/A                           
Hate-unfairness      | 0.0%           | 0.0%                         | N/A                             | N/A                           

Detailed results available at:
https://ai.azure.com

### Interpreting the Basic Scan
Key metric: **Attack Success Rate (ASR)** = percentage of prompts whose responses are flagged harmful.

Because our callback always returns a fixed safe sentence, expected ASR ≈ 0%.

Use the emitted JSON to verify structure and plan which additional strategies or categories to add next.

In [9]:
# Cell 9 - Intermediary Scan (updated to mirror Cell 8 writable path pattern and add basic error handling)
from datetime import datetime, timezone
from pathlib import Path
import pathlib, os

# Lazy client creation (safe on reruns)
if '_rt_client' not in globals():
    from openai import AzureOpenAI
    _rt_client = AzureOpenAI(
        api_key=azure_openai_api_key,
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
    )

# Accept both invocation styles used internally by RedTeam
def model_target(query: str = None, user_prompt: str = None):
    prompt = user_prompt if (user_prompt not in (None, "")) else query or "(empty prompt)"
    return _rt_client.chat.completions.create(
        model=azure_openai_deployment,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=256,
    ).choices[0].message.content or ""

# Reuse writable directory logic from Cell 8 for consistency
_writable_base = pathlib.Path(getattr(red_team.file_manager, 'base_output_dir', Path.home()))
_writable_base.mkdir(parents=True, exist_ok=True)

scan_ts = datetime.now(timezone.utc).strftime('%H%M%S')
scan_label = "Intermediary"
scan_name = f"{scan_label}-Scan-{scan_ts}"
output_path = _writable_base / f"{scan_label.lower()}_scan-{scan_ts}.json"

print(f"Running {scan_name} -> {output_path}")

try:
    intermediary_scan_result = await red_team.scan(
        target=model_target,
        scan_name=scan_name,
        attack_strategies=[AttackStrategy.Flip],
        output_path=str(output_path),
    )
    print(f"Completed {scan_name} -> {output_path}")
except Exception as e:  # noqa: BLE001
    print(f"ERROR during {scan_name}: {e}")
    raise

Running Intermediary-Scan-204737 -> /home/azureuser/redteam_outputs/intermediary_scan-204737.json
🚀 STARTING RED TEAM SCAN
📂 Output directory: /home/azureuser/redteam_outputs/.scan_Intermediary-Scan-204737_20251016_204737
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/dc8cac90-a254-4bee-aaf4-218d28d3e052?wsid=/subscriptions/e440a65b-7418-4865-9821-88e411ffdd5b/resourceGroups/AIRT15/providers/Microsoft.MachineLearningServices/workspaces/AIRT15-project
📋 Planning 4 total tasks
📝 Fetched baseline objectives for violence: 1 objectives
📝 Fetched baseline objectives for hate_unfairness: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


Scanning:   0%|                                       | 0/4 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 4 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: flip strategy for violence risk category
▶️ Starting task: flip strategy for hate_unfairness risk category


Scanning:  75%|███████████████████████▎       | 3/4 [00:23<00:09,  9.79s/scan, current=initializing]

Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Intermediary-Scan-204737_20251016_204737/flip_hate_unfairness_b223034d-f8b8-42d2-9d5f-443c78072e1a.json".
✅ Completed task 1/4 (25.0%) - flip/hate_unfairness in 23.6s
   Est. remaining: 1.2 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Intermediary-Scan-204737_20251016_204737/baseline_hate_unfairness_e983fb80-e67a-4672-a9d9-72bdbe06c8df.json".
✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 23.7s
   Est. remaining: 0.4 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Intermediary-Scan-204737_20251016_204737/flip_violence_06abb1b9-7ebb-43f0-85b5-6971603a02ac.json".
✅ Completed task 3/4 (75.0%) - flip/violence in 23.7s
   Est. remaining: 0.1 minutes


Scanning: 100%|███████████████████████████████| 4/4 [00:23<00:00,  5.97s/scan, current=initializing]


Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Intermediary-Scan-204737_20251016_204737/baseline_violence_b1452ada-d2a2-4b16-83fc-039f3dbef080.json".
✅ Completed task 4/4 (100.0%) - baseline/violence in 23.9s
   Est. remaining: 0.0 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/intermediary_scan-204737.json".

Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Intermediary-Scan-204737_20251016_204737/final_results.json".

Overall ASR: 50.0%
Attack Success: 2/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence             | 0.0%           | 100.0%   

### Moving to a Model Target
Switching from a deterministic callback to an actual model introduces variability and real guardrail evaluation. Keeping the same single `Flip` strategy isolates model safety behavior from added obfuscation complexity.

Next expansions: increase `num_objectives`, add additional strategies (CharSwap, UnicodeConfusable, etc.), or broaden risk categories.

In [10]:
# Cell 10: Advanced scan - create expanded RedTeam instance (with writable base_output_dir)
advanced_risk_categories = [
    RiskCategory.Violence,
    RiskCategory.HateUnfairness,
    RiskCategory.Sexual,
    RiskCategory.SelfHarm,
]
advanced_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=advanced_risk_categories,
    num_objectives=3,  # increase coverage per category (adjust for cost/time)
)

# Mirror writable directory logic used earlier for 'red_team'
import pathlib
_adv_out = pathlib.Path(getattr(red_team.file_manager, 'base_output_dir', pathlib.Path.home()))
_adv_out.mkdir(parents=True, exist_ok=True)
try:
    if hasattr(advanced_red_team, 'file_manager') and hasattr(advanced_red_team.file_manager, 'base_output_dir'):
        advanced_red_team.file_manager.base_output_dir = str(_adv_out)
        print('Advanced RedTeam base_output_dir set to', advanced_red_team.file_manager.base_output_dir)
    else:
        print('WARNING: Could not set advanced_red_team base_output_dir; attribute not found')
except Exception as _e:
    print('WARNING: Failed setting advanced_red_team base_output_dir:', _e)

print("Advanced RedTeam configured with categories:", [c.name for c in advanced_risk_categories])

Advanced RedTeam base_output_dir set to /home/azureuser/redteam_outputs
Advanced RedTeam configured with categories: ['Violence', 'HateUnfairness', 'Sexual', 'SelfHarm']


### Expanding Coverage
We now include all four core risk categories and raise `num_objectives` to increase statistical signal. This increases token/time consumption proportionally.

Broader coverage helps surface category-specific weaknesses early (e.g., higher ASR in SelfHarm vs Sexual).

In [11]:
# Cell 11: Advanced scan (standardized naming & writable output dir)
from datetime import datetime, timezone
from pathlib import Path
import pathlib  # for consistent writable directory handling

scan_ts = datetime.now(timezone.utc).strftime('%H%M%S')
scan_label = "Advanced"
scan_name = f"{scan_label}-Scan-{scan_ts}"

# Use same writable directory pattern as earlier scans (fallback to home if attribute missing)
_writable_base = pathlib.Path(getattr(red_team.file_manager, 'base_output_dir', Path.home()))
_writable_base.mkdir(parents=True, exist_ok=True)
output_path = _writable_base / f"{scan_label.lower()}_scan-{scan_ts}.json"

print(f"Running {scan_name} -> {output_path}")
advanced_scan_result = await advanced_red_team.scan(
    target=model_target,
    scan_name=scan_name,
    attack_strategies=[
        AttackStrategy.EASY,
        AttackStrategy.MODERATE,
        AttackStrategy.Flip,
        AttackStrategy.CharSwap,
        AttackStrategy.UnicodeConfusable,
        AttackStrategy.Leetspeak,
        AttackStrategy.Url,
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),
    ],
    output_path=str(output_path),
)
print(f"Completed {scan_name} -> {output_path}")

Running Advanced-Scan-205004 -> /home/azureuser/redteam_outputs/advanced_scan-205004.json
🚀 STARTING RED TEAM SCAN
📂 Output directory: /home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/61b1d8e2-6346-48b1-bb75-0d72493ce006?wsid=/subscriptions/e440a65b-7418-4865-9821-88e411ffdd5b/resourceGroups/AIRT15/providers/Microsoft.MachineLearningServices/workspaces/AIRT15-project
📋 Planning 40 total tasks
📝 Fetched baseline objectives for violence: 3 objectives
📝 Fetched baseline objectives for hate_unfairness: 3 objectives
📝 Fetched baseline objectives for sexual: 3 objectives
📝 Fetched baseline objectives for self_harm: 3 objectives
🔄 Fetching objectives for strategy 2/10: flip
🔄 Fetching objectives for strategy 3/10: char_swap
🔄 Fetching objectives for strategy 4/10: unicode_confusable
🔄 Fetching objectives for strateg

Scanning:   0%|                                      | 0/40 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 40 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for self_harm risk category
▶️ Starting task: flip strategy for violence risk category


Scanning:   8%|██▎                           | 3/40 [01:36<59:11, 95.99s/scan, current=initializing]

Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/baseline_violence_e02b0b85-d12f-4c80-b12b-a86d190fba2d.json".
✅ Completed task 1/40 (2.5%) - baseline/violence in 96.0s
   Est. remaining: 63.2 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/baseline_self_harm_c022c2d3-2a88-44d5-a075-07aa62cfcd8a.json".
✅ Completed task 2/40 (5.0%) - baseline/self_harm in 96.0s
   Est. remaining: 30.8 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/baseline_hate_unfairness_6e1ac2cc-c01e-4cd8-a5f9-da8da8b01c94.json".
✅ Completed task 3/40 (7.5%) - baseline/hate_unfairness in 96.0s
   Est. remaining: 20.0 minutes


...

Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/base64_self_harm_60cc2a7e-9b8b-4dfe-8f2a-734ab64e6020.json".
✅ Completed task 34/40 (85.0%) - base64/self_harm in 104.0s
   Est. remaining: 2.2 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/morse_hate_unfairness_4e7a3efd-7355-4771-964f-1179e4ec784f.json".
✅ Completed task 35/40 (87.5%) - morse/hate_unfairness in 104.0s
   Est. remaining: 1.8 minutes
▶️ Starting task: morse strategy for self_harm risk category
▶️ Starting task: tense strategy for violence risk category
▶️ Starting task: tense strategy for hate_unfairness risk category
▶️ Starting task: tense strategy for sexual risk category
▶️ Starting task: tense strategy for self_harm risk category


ERROR: [tense/violence] Error processing prompts: Converted prompt text is None.
Traceback (most recent call last):
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/azure/ai/evaluation/red_team/_orchestrator_manager.py", line 264, in _prompt_sending_orchestrator
    await send_all_with_retry()
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/tenacity/asyncio/__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/tenacity/asyncio/__init__.py", line 111, in __call__
    do = await self.iter(retry_state=retry_state)
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/tenacity/asyncio/__init__.py", line 153, in iter
    result = await action(retry_state)
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/tenacity/_utils.py", line 99, in inner
    return call(*args, **kwargs)
  File "/anaconda/envs/jupyter_env/lib/python3.10/site-packages/tenaci

Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/tense_self_harm_3bb107a5-a7b3-4612-934a-c2734863d3a9.json".
✅ Completed task 36/40 (90.0%) - tense/self_harm in 115.7s
   Est. remaining: 1.6 minutes


Scanning: 100%|█████████████████████████████| 40/40 [14:31<00:00, 21.79s/scan, current=initializing]


Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/tense_violence_20629c0c-1752-4f96-bc02-5743651b33fe.json".
✅ Completed task 37/40 (92.5%) - tense/violence in 123.7s
   Est. remaining: 1.2 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/morse_self_harm_733a5ea2-223b-42af-921f-fa1d011d53ae.json".
✅ Completed task 38/40 (95.0%) - morse/self_harm in 123.7s
   Est. remaining: 0.8 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/tense_hate_unfairness_76e15fbd-cc20-42d5-a2f8-0c510dbd3d52.json".
✅ Completed task 39/40 (97.5%) - tense/hate_unfairness in 123.7s
   Est. remaining: 0.4 minutes
Evaluation results saved to "/home/azureuser/redteam_outputs/.scan_Advanced-Scan-205004_20251016_205004/tense_sexual_ee522035-8497-4107-a86a-5e06b97d5c75.json".
✅ Completed task 40/40 (100.0%) - tense/sexual in 123.7s
   Est. re

### Advanced Strategies & Layering
The advanced scan mixes:
- Complexity groups (`EASY`, `MODERATE`) for breadth.
- Obfuscations (CharSwap, UnicodeConfusable, Leetspeak, Url) to probe normalization defenses.
- Encoding (Base64, ROT13 via composition) to test decoding / content safety layers.

Capturing stderr lets you quickly surface any internal SDK errors alongside scan results.

## Bring Your Own Objectives: Custom Attack Seed Prompts
You can supply your own domain or application-specific risky prompts as objectives instead of (or in addition to) automatically generated ones.

Format: a JSON file whose entries include `prompt` text and `risk-type` (one of: `violence`, `sexual`, `hate_unfairness`, `self_harm`). The number of prompts provided becomes the effective `num_objectives` for the scan.

Use this when:
- You have proprietary misuse scenarios not covered by generic seeds.
- You want regression tracking on a fixed, curated risky prompt set.
- You need to validate mitigations against previously successful attacks.

Below we instantiate a new `RedTeam` with `custom_attack_seed_prompts` pointing to `data/prompts.json`, then run grouped difficulty strategies.

> Tip: Keep a version-controlled prompts file so additions are reviewable and diffs tie to shifts in ASR.


In [ ]:
# Cell 12 - Custom prompts RedTeam instance (dynamic prompt discovery + writable dir)
from pathlib import Path
import pathlib, os, re

# Allow explicit override
explicit_prompt_path = os.environ.get("REDTEAM_CUSTOM_PROMPTS_PATH")

candidates = []
searched = []
selected = None

if explicit_prompt_path:
    p = Path(explicit_prompt_path)
    if p.is_file():
        candidates.append(p)

# Derive project mount pattern similar to Cell 4 logic: /afh/projects/*-project-*/shared/files/data/prompts.json
projects_root = Path('/afh/projects')
if projects_root.is_dir():
    for child in projects_root.iterdir():
        if child.is_dir() and ('-project-' in child.name):
            prompt_candidate = child / 'shared' / 'files' / 'data' / 'prompts.json'
            searched.append(prompt_candidate)
            if prompt_candidate.is_file():
                candidates.append(prompt_candidate)

# Fallback: glob search for any prompts.json under shared/files/data
if not candidates and projects_root.is_dir():
    for prompt_candidate in projects_root.glob('**/shared/files/data/prompts.json'):
        searched.append(prompt_candidate)
        if prompt_candidate.is_file():
            candidates.append(prompt_candidate)
            break

# Select first existing candidate
for c in candidates:
    try:
        if c.is_file():
            selected = c
            break
    except Exception:
        pass

if not selected:
    raise FileNotFoundError(
        "Could not locate prompts.json. Searched candidates (first 5): " +
        str([str(p) for p in searched[:5]]) +
        " - set REDTEAM_CUSTOM_PROMPTS_PATH to override."
    )

print(f"Using prompts file: {selected}")

custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=str(selected),
)

# Align writable output directory with earlier scans
_custom_out = pathlib.Path(getattr(red_team.file_manager, 'base_output_dir', Path.home()))
_custom_out.mkdir(parents=True, exist_ok=True)
try:
    if hasattr(custom_red_team, 'file_manager') and hasattr(custom_red_team.file_manager, 'base_output_dir'):
        custom_red_team.file_manager.base_output_dir = str(_custom_out)
        print('Custom RedTeam base_output_dir set to', custom_red_team.file_manager.base_output_dir)
    else:
        print('WARNING: Could not set custom_red_team base_output_dir; attribute not found')
except Exception as _e:
    print('WARNING: Failed setting custom_red_team base_output_dir:', _e)

print("Custom RedTeam ready. Prompt count determines num_objectives.")

In [ ]:
# Cell 13: Execute scan with custom prompts and grouped difficulty strategies (writable output dir)
from datetime import datetime, timezone
from pathlib import Path
import pathlib

scan_ts = datetime.now(timezone.utc).strftime('%H%M%S')
scan_label = "Custom"
scan_name = f"{scan_label}-Prompt-Scan-{scan_ts}"

# Use the custom_red_team writable directory (fallback to home)
_writable_base = pathlib.Path(getattr(custom_red_team.file_manager, 'base_output_dir', Path.home()))
_writable_base.mkdir(parents=True, exist_ok=True)
output_path = _writable_base / f"custom_prompt_scan-{scan_ts}.json"
print(f"Running {scan_name} -> {output_path}")

try:
    custom_result = await custom_red_team.scan(
        target=model_target,  # reuse earlier model target callback
        scan_name=scan_name,
        attack_strategies=[
            AttackStrategy.EASY,
            AttackStrategy.MODERATE,
            AttackStrategy.DIFFICULT,
        ],
        output_path=str(output_path),
    )
    print(f"Completed {scan_name} -> {output_path}")
except Exception as e:  # noqa: BLE001
    print(f"ERROR during {scan_name}: {e}")
    raise